Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel $\rightarrow$ Restart) and then **run all cells** (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = ""
COLLABORATORS = ""

---

# Домашнее задание №4. Матричная факторизация

## Задачи - 25 баллов (+5 доп баллов)
1. Ноутбук `mf.ipynb` - 20 баллов
- ImplicitALS - 4 балла
- SVD - 4 балла
- Dataset with features - 2 балла
- ImplicitALS with features - 5 баллов
- LightFM with features - 5 баллов
2. Имплементация модели в сервис - 5 баллов
- Пробить на Leaderboard порог `map@10 = 0.075`
- Если при этом используете MF (Implicit или LightFM) + ANN (nmslib, faiss, annoy и тд) - дополнительно 5 баллов
  
## Как сдать ноутбук `mf.ipynb` на проверку

1. Прогоните весь код ноутбука - проверьте, что нет ошибок и тесты проходят
2. Выложите готовый ноутбук в ваш репозиторий с сервисом из домашнего задания №1 по пути `notebooks/hw_4/mf.ipynb` в ветке `hw_4`
3. Проверьте, что есть доступ к вашему репозиторию для аккаунтов `https://github.com/feldlime`
4. Откройте PR в main ветку и добавьте в ревьюеры **своего ментора**
5. Не проводите мердж в `main` ветку, пока не увидите оценку за это ДЗ в ведомости. Файл с ноутбуком должен находиться в ветке `hw_4`

Обратите внимание, что сборка ноутбуков на проверку автоматизирована. В случае неправильного пути, имени файла или ветки (а также при отсутствии доступа у `@feldlime`) ваша работа не попадёт на проверку и получит `0` баллов.

Используемые библиотеки в рамках ДЗ
```bash
pip install implicit==0.7.2 requests==2.32.3 rectools[lightfm]==0.12.0 pandas==2.2.3 numpy==1.26.4 scipy==1.12.0
```

In [ ]:
!pip install implicit==0.7.2 requests==2.32.3 rectools[lightfm]==0.12.0 pandas==2.2.2 numpy==1.26.4 scipy==1.12.0 optuna

## Импорты и данные

In [ ]:
import os
import threadpoolctl
import requests

import numpy as np
import pandas as pd
import zipfile as zf

from tqdm.auto import tqdm
from implicit.als import AlternatingLeastSquares
from lightfm import LightFM

from rectools import Columns
from rectools.metrics import MAP, MeanInvUserFreq
from rectools.dataset import Dataset
from rectools.models import PureSVDModel, ImplicitALSWrapperModel, LightFMWrapperModel, model_from_config

# For implicit ALS
os.environ["OPENBLAS_NUM_THREADS"] = "1"
threadpoolctl.threadpool_limits(1, "blas")

Если вдруг у вас нет данных, то используйте закомментированный код

In [ ]:
# url = 'https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip'

# req = requests.get(url, stream=True)

# with open('kion.zip', 'wb') as fd:
#     total_size_in_bytes = int(req.headers.get('Content-Length', 0))
#     progress_bar = tqdm(desc='kion dataset download', total=total_size_in_bytes, unit='iB', unit_scale=True)
#     for chunk in req.iter_content(chunk_size=2 ** 20):
#         progress_bar.update(len(chunk))
#         fd.write(chunk)

# files = zf.ZipFile('kion.zip', 'r')
# files.extractall()
# files.close()

In [ ]:
url = 'https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip'

req = requests.get(url, stream=True)

with open('kion.zip', 'wb') as fd:
    total_size_in_bytes = int(req.headers.get('Content-Length', 0))
    progress_bar = tqdm(desc='kion dataset download', total=total_size_in_bytes, unit='iB', unit_scale=True)
    for chunk in req.iter_content(chunk_size=2 ** 20):
        progress_bar.update(len(chunk))
        fd.write(chunk)

files = zf.ZipFile('kion.zip', 'r')
files.extractall()
files.close()

In [ ]:
interactions = (
    pd.read_csv('data_original/interactions.csv', parse_dates=["last_watch_dt"])
    .rename(columns={'total_dur': Columns.Weight,
                     'last_watch_dt': Columns.Datetime})
)
users = pd.read_csv('data_original/users.csv')
items = pd.read_csv('data_original/items.csv')

print(interactions.shape)
interactions.head(5)

In [ ]:
N_DAYS = 7

max_date = interactions['datetime'].max()
train = interactions[(interactions['datetime'] <= max_date - pd.Timedelta(days=N_DAYS))]
test = interactions[(interactions['datetime'] > max_date - pd.Timedelta(days=N_DAYS))]

catalog = train[Columns.Item].unique()

test_users = test[Columns.User].unique()
cold_users = set(test_users) - set(train[Columns.User])
test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)
hot_users = test[Columns.User].unique()

# dataset = Dataset.construct(train)

In [ ]:
K_RECOS = 10
map10 = MAP(k=K_RECOS)

## ImplicitALS

### Ситуация:

Коллега вернулся из отпуска и вы вместе сели за улучшение модели. Внимательно изучив репозиторий библиотеки implicit вы увидели модель iALS и решаете попробовать ее в деле.

Чтобы работа была интереснее, вы заключаете пари с вашим коллегой о том, кто выбьет больше MAP@K на горячих пользователях.

Правила пари:
- Валидируемся на последней неделе (переменная `test`) и на горячих пользователях `hot_users`
- Можно собрать свой `Dataset` на основе `train`, трансформированного, если нужно
- Параметры модели задаются конфигом, которые будут передаваться в `model_from_config`

У вашего коллеги получилось выбить на ImplicitALS `MAP@K = 0.052`. Ваша задача побить его рекорд.

In [ ]:
# train preprocessing...
train2 = train.copy()
train2[Columns.Weight] = np.where(train2['watched_pct'] > 10, 3, 1)
test[Columns.Weight] = np.where(test['watched_pct'] > 10, 3, 1)

dataset = Dataset.construct(train2)

In [ ]:
config = {
    'cls': 'ImplicitALSWrapperModel',
    'verbose': 1,
    'model': {
        'factors': 8,
        'regularization': 0.17,
        'iterations': 25,
        'use_gpu': False,
        'alpha': 15.0,
        'random_state': 42,
        'num_threads': 4
    }
}

In [ ]:
%%time
assert config['cls'] == 'ImplicitALSWrapperModel'

model = model_from_config(config)
model.fit(dataset)

recos = model.recommend(
    users=hot_users,
    dataset=dataset,
    k=K_RECOS,
    filter_viewed=True,
)
print(map10.calc(recos, test))

assert map10.calc(recos, test) >= 0.052

## SVD

На ваш громкий спор с коллегой о том, что все дело в вашем удачном random seed, к вам подошел ваш лид.

Узнав детали вашего спора, он дает вам комментарий, что iALS хороша, но погружение в матричную факторизацию следует начинать с `SVD`.

Вы переглянулись с коллегой и решаете уладить спор о random seed во втором раунде, используя новую модель.

Ваш коллега смогу выбить на SVD `MAP@K = 0.066`. Вы знаете, что делать.

In [ ]:
# train preprocessing...
train2 = train.copy()
train2[Columns.Weight] = np.where(train2['watched_pct'] > 10, 3, 1)
test[Columns.Weight] = np.where(test['watched_pct'] > 10, 3, 1)

dataset2 = Dataset.construct(train2)

In [ ]:
config = {
    'cls': 'PureSVDModel',
    "factors": 1,
    "tol": 2.1e-05,
    "maxiter": 58,
    "random_state": 42,
    "use_gpu": False,
    "verbose": 0,
    "recommend_n_threads": 4,
    "recommend_use_gpu_ranking": False
}

In [ ]:
%%time
assert config['cls'] == 'PureSVDModel'

model = model_from_config(config)
model.fit(dataset2)

recos = model.recommend(
    users=hot_users,
    dataset=dataset2,
    k=K_RECOS,
    filter_viewed=True,
)
print(map10.calc(recos, test))

assert map10.calc(recos, test) >= 0.066

## Dataset with features

"Ну это ни в какие ворота!" - восклицает ваш коллега, увидев ваш победный конфиг. Из другого угла опенспейса доносится "А я говорил" от вашего лида.

В это время к вам сзади подходит продакт и интересуется предметом вашего спора.

Рассказав про особенности найденных вами моделей, он просит вас в них докинуть фичи, ведь на одних взаимодействиях далеко не уедешь.

Вы согласились, ведь это отличная возможность продолжить пари. Соберите `Dataset` с фичами по пользователям и итемам, который вы будете использовать дальше


In [ ]:
user_features_frames = []
for feature in ["sex", "age", "income"]:
    feature_frame = users.reindex(columns=[Columns.User, feature])
    feature_frame.columns = ["id", "value"]
    feature_frame["feature"] = feature
    user_features_frames.append(feature_frame)
user_features = pd.concat(user_features_frames)

In [ ]:
items["genre"] = items["genres"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
genre_feature = items[["item_id", "genre"]].explode("genre")
genre_feature.columns = ["id", "value"]
genre_feature["feature"] = "genre"

content_feature = items.reindex(columns=[Columns.Item, "content_type"])
content_feature.columns = ["id", "value"]
content_feature["feature"] = "content_type"

age_rating = items.reindex(columns=[Columns.Item, "age_rating"])
age_rating.columns = ["id", "value"]
age_rating["feature"] = "age_rating"

item_features = pd.concat((genre_feature, content_feature, age_rating))

In [ ]:
dataset_with_features = Dataset.construct(
    interactions_df=train2,
      user_features_df=user_features,
    cat_user_features=["sex", "age", "income"],
    item_features_df=item_features,
    cat_item_features=["genre", "content_type", "age_rating"]
    )

In [ ]:
assert (dataset_with_features.user_features is not None) and (dataset_with_features.item_features is not None)

## ImplicitALS with features

Собрав датасет с фичами вы готовы к третьему раунду пари.

Вы решаете начать снова с `iALS`, до сих пор удивляясь результатам модели `SVD`.

Ваш коллега изучил вашу технику подбора random seed и хитро улыбается вам.

Он смог выбить `MAP@K = 0.073`, теперь ваш ход.

In [ ]:
config = {
    "cls": "ImplicitALSWrapperModel",
    "verbose": 1,
    "model": {
      "factors": 1,
      "regularization": 0.005,
      "alpha": 40,
      "dtype": np.float32,
      "use_native": True,
      "use_cg": True,
      "use_gpu": False,
      "iterations": 15,
      "calculate_training_loss": False,
      "num_threads": 4,
      "random_state": 42,
    }
}

In [ ]:
%%time
assert config['cls'] == 'ImplicitALSWrapperModel'

model = model_from_config(config)
model.fit(dataset_with_features)

recos = model.recommend(
    users=hot_users,
    dataset=dataset_with_features,
    k=K_RECOS,
    filter_viewed=True,
)
print(map10.calc(recos, test))

assert map10.calc(recos, test) >= 0.073

## LightFM with features

И снова ор выше гор, ваш пайплайн подготовки датасета помог вам в очередной раз обойти вашего коллегу.

Не зная, к чему еще аппелировать, он зовет вашего старшего коллегу, чтобы тот внимательно изучил полученные результаты.

"iALS с фичами это хорошо, но тут стоит попробовать факторизационные машины, попробуйте `LightFM`" - заключает он. Вы переключаетесь на изучение новой библиотеки, предвкушая финальный раунд.

Ваш коллега смог выжать из своего обновленного `Dataset` и `LightFM` скор `MAP@10 = 0.08`. Последний рывок.

In [ ]:
config = {
    "cls": 'LightFMWrapperModel',
    'verbose': 1,
    'model':     {
        "no_components": 1,
        "k": 4,
        "learning_schedule": "adagrad",
        "loss": "warp-kos",
        "learning_rate": 0.04,
        "item_alpha": 4.44008526355965e-06,
        "user_alpha": 1.1899614693656712e-07,
        "max_sampled": 20,
        "random_state": 42
        }
}

In [ ]:
%%time
assert config['cls'] == 'LightFMWrapperModel'

model = model_from_config(config)
model.fit(dataset_with_features)

recos = model.recommend(
    users=hot_users,
    dataset=dataset_with_features,
    k=K_RECOS,
    filter_viewed=True,
)
print(map10.calc(recos, test))

assert map10.calc(recos, test) >= 0.08

## Сервис

Эта битва была легендарной, но вот за спиной опять появляется ваш продакт.

"Время катить АБ" - говорит он в пятницу вечером. Делать нечего, вы собираете ваши наработки и определяете совместный фронт работ.

В прод должна заехать лучшая модель, которая побьет текущей модели в проде в `MAP@10 = 0.075`.

Также есть бонус от вашего старшего коллеги, который вам советует присмотреться к `Approximate nearest neighbours`, например к `nmslib`.

"Если сможешь обернуть в ANN, то на следующем годовом ревью получишь от меня оценку отлично" - сказал он. Изучить новые технологии и получить повышение - идеально, заключаете вы и бросаетесь в бой.

# Вместо заключения

## Задачи - 25 баллов (30 баллов с доп задачей по ANN)
1. Ноутбук `mf.ipynb` - 20 баллов
- SVD - 5 баллов
- ImplicitALS - 5 баллов
- ImplicitALS with features - 5 баллов
- LightFM with features - 5 баллов
2. Имплементация модели в сервис - 5 баллов
- Пробить на Leaderboard порог `map@10 = 0.075`
- Если при этом используете MF (Implicit или LightFM) + ANN (nmslib, faiss, annoy и тд) - дополнительно 5 баллов
  
## Как сдать ноутбук `mf.ipynb` на проверку

1. Прогоните весь код ноутбука - проверьте, что нет ошибок и тесты проходят
2. Выложите готовый ноутбук в ваш репозиторий с сервисом из домашнего задания №1 по пути `notebooks/hw_4/mf.ipynb` в ветке `hw_4`
3. Проверьте, что есть доступ к вашему репозиторию для аккаунтов `https://github.com/feldlime`
4. Откройте PR в main ветку и добавьте в ревьюеры **своего ментора**
5. Не проводите мердж в `main` ветку, пока не увидите оценку за это ДЗ в ведомости. Файл с ноутбуком должен находиться в ветке `hw_4`

Обратите внимание, что сборка ноутбуков на проверку автоматизирована. В случае неправильного пути, имени файла или ветки (а также при отсутствии доступа у `@feldlime`) ваша работа не попадёт на проверку и получит `0` баллов.